In [1]:
from google.colab import userdata

# import token
hf_auth = userdata.get('HF_TOKEN')

lm_model_base = 'meta-llama/Llama-3.2-1B'
lm_model_inst = 'meta-llama/Llama-3.2-1B-Instruct'

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

tokenizer = AutoTokenizer.from_pretrained(lm_model_base, token=hf_auth)
model = AutoModelForCausalLM.from_pretrained(lm_model_base, token=hf_auth)
device = 'cuda'
model.to(device)

In [4]:
question = 'what is the best EDM track of all the times?'
tokenizer.decode(tokenizer(question)['input_ids'])

'<|begin_of_text|>what is the best EDM track of all the times?'

In [5]:
# maggiori dettagli per text generation
# https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/text_generation#transformers.GenerationConfig

def ask(question, model, device):

    tk = tokenizer(question, return_tensors='pt')
    tk['input_ids'] = tk['input_ids'].to(device)
    tk['attention_mask'] = tk['attention_mask'].to(device)

    gen_config = GenerationConfig(
        do_sample=True,
        max_new_tokens=256,
        temperature=0.0000001)

    response = model.generate(
        input_ids=tk['input_ids'],
        attention_mask=tk['attention_mask'],
        generation_config=gen_config)

    # per effettuare il parsing e ottenere solo la risposta e non prompt+input+answer
    answer = tokenizer.batch_decode(response[:, len(tk['input_ids'][0]):], skip_special_tokens=True)[0]

    return response, answer


In [6]:
response, answer= ask(question, model, device)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])
print('-----')
print(answer)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(lm_model_inst)
model = AutoModelForCausalLM.from_pretrained(lm_model_inst)
device = 'cuda'
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [ ]:
# inst
response = ask(question, model, device)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


what is the best EDM track of all the times? (subjective opinion)
I'm a huge fan of EDM and I've been listening to it for years. I've heard a lot of great tracks, but I'm still trying to find the one that stands out from the rest. After some research and listening to a wide variety of EDM tracks, I've narrowed it down to a few contenders. Here are my top 5 contenders for the best EDM track of all time:

**1. "Sandstorm" by Darude (1999)**
This Finnish trance track is a masterclass in building tension and release. The driving beat, soaring synths, and epic drop make it a timeless classic. It's a staple of the genre and a favorite among DJs and producers.

**2. "One" by Veracocha (1999)**
This French trance track is a staple of the early 2000s. The catchy melody, driving beat, and epic drop make it a standout track. It's a great example of the genre's ability to create infectious, dancefloor-friendly tracks.

**3. "Insomnia" by Faithless (1995)**
This British trance track is a classic of

In [ ]:
# cosa succede con un modello instruct quando forniamo più dettagli nel prompt?
# come cambia la risposta del modello?

messages1 = [
    {"role": "system", "content": "Answer kindly to the following question or request"},
    #{"role": "system", "content": "Answer kindly to the following question or request as you are a dj on a famous radio program"},
    #{"role": "system", "content": "Answer kindly to the following question or request as you are presenting the Grammy Awards"},
    #{"role": "system", "content": "Answer kindly to the following question or request as you are a news' host"},
    {"role": "user", "content": "What are the best EDM tracks of all the 10s"},
    {"role": "user", "content": "What are the best EDM tracks of all the 10s"},
]

print(tokenizer.apply_chat_template(messages1, tokenize=False))

response = ask(tokenizer.apply_chat_template(messages1, tokenize=False), model, device)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 20 Nov 2024\n\nAnswer kindly to the following question or request<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat are the best EDM tracks of all the 10s<|eot_id|>'

In [ ]:
def ask(pipe, messages):
    outputs = pipe(
        messages,
        temperature=0.0000001,
        max_new_tokens=512,
        #generation_config=gen_config
    )

    answer = outputs[0]["generated_text"][-1]['content']
    print(messages[0]['content'])
    print(messages[1]['content'])
    print(answer)

    return answer

In [ ]:
# pipeline di hugging face embedded per text generation
from transformers import pipeline

pipe = pipeline(
        "text-generation",
        model=lm_model_inst,
        token = hf_auth,
        device_map='cuda',
    )

messages1 = [
    {"role": "system", "content": "Answer kindly to the following question or request"},
    {"role": "user", "content": "What are the best EDM tracks of all the 10s"},
]
answer = ask(pipe, messages1)